<a href="https://colab.research.google.com/github/DibyaSadhukhan/Portfolio/blob/main/main/Webscraping/Moneycontrol/Money_control_stock_data_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#importing required packages
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import random
import time

In [3]:
def allele(lists):
    #since the entries are a list of lists 
    #this function takes in a list of lists and returns the elements in the list of list in a list of dimension (n,1)
    ele=[]
    for elements in reversed(lists):
        for element in elements:
            ele.append(element)
    return ele
def urlgenpar(year,bigmonth,endmonth):
    #since the data begins from april 2007 so we need a function to generate the url for years with less than 12 months
    #this function takes in the year the beginning month and end months and returns a list of url 
    url=[]
    for month in range(bigmonth,endmonth+1):
        #since the url have 04 05 for the months of april etc we need to add the 0 externally
        if month<10:
            url.append('https://www.moneycontrol.com/techmvc/responsive/fiidii/monthly?month=0'+str(month)+'&year='+str(year)+'&section=cash&sub_section=')
        else:
            url.append('https://www.moneycontrol.com/techmvc/responsive/fiidii/monthly?month='+str(month)+'&year='+str(year)+'&section=cash&sub_section=')
    return url
def urlgenfull(year):
    #this function creates a url list of the entirity of the year provided
    #this function takes in the year as a parameter and returns the generated url list as the output 
    url=[]
    for month in range(1,13):
        if month<10:
            #since the url have 04 05 for the months of april etc we need to add the 0 externally
            url.append('https://www.moneycontrol.com/techmvc/responsive/fiidii/monthly?month=0'+str(month)+'&year='+str(year)+'&section=cash&sub_section=')
        else:
            url.append('https://www.moneycontrol.com/techmvc/responsive/fiidii/monthly?month='+str(month)+'&year='+str(year)+'&section=cash&sub_section=')
    return url



In [4]:
def datawriter(url):
    #this function writes the data for the entire year and reshapes it into the dimensions of the table.
    #this function takes in the url list as a paramenter and returns a (Nx7) array as an output
    data=[]
    for base in url: 
        #opening every individual urls to extract their data
        headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
        webPage = requests.post(base, headers = headers).text
        soup = BeautifulSoup(webPage, 'lxml')
        text=[]
        #td tags also have the column names so we skipp the first 3 elements of the list
        for tag in soup.find_all('td'):
            text.append(tag.text.strip())
        data.append(text[3:])
        time.sleep(random.randint(1,2))
    #changging the dimension of the list from list of lists to (nx1)
    data=allele(data)
    data.reverse()
    #print(data)
    arr = np.array(data)
    #since there are 7 rows and we dont have missing data we divide the total length by 7 and make it into a (nx7) matrix
    arr = arr.reshape(int(len(arr)/7),7)
    return arr
def createDirectory(directory):
  import os.path
  from os import path
  if path.exists(directory) == False:
    os.mkdir(directory)
    print("Created :",directory)
  else:
    print("Directory already exists")
def writedatapar(year,bigmonth,endmonth):
    #this function writes the data for the partial year and appends it to the stock data csv.
    #this function takes in the year with the diffent months as theparamenter and writes it in the stock data csv
    url=urlgenpar(year,bigmonth,endmonth)
    arr=datawriter(url)
    print("Writing file...")
    df2 = pd.DataFrame(arr) 
    createDirectory('/content/Data')
    try:
        #if the stockdata.csv exists open it
        path='Data/StockData.csv'
        df = pd.read_csv(path,header=None)

    except:
        #if it throws error create a csv with two rows of 7 columns with 0s in them which can be removed later
        df = pd.DataFrame([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        df.to_csv('Data/StockData.csv',index=False)
        path='Data/StockData.csv'
        df = pd.read_csv(path,header=None)
    df=df.append(df2)

    df.to_csv('Data/StockData.csv',index=False)

def writedatafull(year):
    #this function writes the data for the full year and appends it to the stock data csv.
    #this function takes in the year with the diffent months as theparamenter and writes it in the stock data csv
    url=urlgenfull(year)
    arr=datawriter(url)
    df2 = pd.DataFrame(arr) 
    createDirectory('/content/Data')
    try:
        path='Data/StockData.csv'
        df = pd.read_csv(path,header=None)
    except:
        df = pd.DataFrame([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])
        df.to_csv('Data/StockData.csv',index=False)
        path='Data/StockData.csv'
        df = pd.read_csv(path,header=None)
    df=df.append(df2)
    df.to_csv('Data/StockData.csv',index=False)
#writedatapar(2007,4,12)
for i in range(2008,2022):
    writedatafull(i)
print("data saved at /Data/StockData.csv" )
#print(df.head(10))
#df.to_csv('Data/StockData.csv',index=False)

Created : /content/Data
Directory already exists
Directory already exists
Directory already exists
Directory already exists
Directory already exists
Directory already exists
Directory already exists
Directory already exists
Directory already exists
Directory already exists
Directory already exists
Directory already exists
Directory already exists
data saved at /Data/StockData.csv


In [3]:
path='Data/StockData.csv'
df = pd.read_csv(path,header=None)
#end year -start year +3
df2=df.iloc[((2022-2008)+3):]
df2.columns=['DII Net purchase/Sales','DII gross Sales','DII Gross purchase','FII Net purchase/Sales','FII gross Sales','FII Gross purchase','Date']
df2.head()

,DII Net purchase/Sales,DII gross Sales,DII Gross purchase,FII Net purchase/Sales,FII gross Sales,FII Gross purchase,Date
17,306.94,977.44,"1,284.38",-461.41,"1,350.10",888.69,01-Jan-2008
18,427.11,"1,183.38","1,610.49",-628.68,"4,426.81","3,798.13",02-Jan-2008
19,526.65,"1,497.85","2,024.50",-244.56,"5,654.54","5,409.98",03-Jan-2008
20,506.85,"1,445.60","1,952.45",16.01,"4,526.29","4,542.30",04-Jan-2008
21,327.64,"1,479.00","1,806.64","-1,543.44","5,475.53","3,932.09",07-Jan-2008


In [4]:
df2.to_csv('Data/StockData.csv',index=False)